In [1]:
import tensorflow
import numpy
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import preprocessing
from tensorflow.keras.layers import  Embedding, Dense, Flatten, LSTM, Dropout, BatchNormalization, Bidirectional, Conv1D, MaxPooling1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import io
import json
import os
import re
import random, sys

In [2]:
text = open('train3.txt').read().lower()


In [10]:
text

'начинается загрузка данных ++загрузитьисторию  а у меня бфр попала тогда? надо крч ордов докачать чтоб мануал в бт не заканчивался на 1-2хп ок ага да скорее всего в бт <:l1zh6l2hdee:683696525896843291> начинается загрузка данных нннн ++загрузитьисторию бля ну примерно хотя бы это тоже не точно показывает смотрю, когда из сети выйдет так-что выбор не велик и что-то никакой киберкотлетной славы я у тебя не вижу а нахуя ты проверяешь статус? начинается загрузка данных ++загрузитьисторию я вообще хз нахуя им она там им не нужна там точная инфа <:im_listening:782713631862685710> соде просто похуй разберись инфу хуёвую даёт сидит там в индии где-то бот просто выводит инфу из апи начинается загрузка данных ++загрузитьисторию я вообще хз нахуя им она там им не нужна там точная инфа <:im_listening:782713631862685710> соде просто похуй разберись инфу хуёвую даёт сидит там в индии где-то бот просто выводит инфу из апи дай безду этому апи причем тут бот это апи <:bez_slez_ne_vzglyanesh:7290985297

In [3]:
test = re.sub(r'<[^()]*>', '', text)
print(len(text) - len(test))
print(len(text))

55696
22436462


In [4]:
text_file = open("generalText.txt", "w")
generalText = text
text_file.write(generalText)
text_file.close()
len(generalText)

22436462

In [5]:
maxlen = 60

step = 3

sentences = []

next_chars = []

for i in range(0, len(generalText) - maxlen, step):
    sentences.append(generalText[i: i + maxlen])
    next_chars.append(generalText[i + maxlen])

print('Number of sequences:', len(sentences))

chars = sorted(list(set(generalText)))
print('Unique characters:', len(chars))

char_indices = dict(((char, chars.index(char)) for char in chars))

x = numpy.zeros((len(sentences), maxlen, len(chars)), dtype=numpy.bool)
y = numpy.zeros((len(sentences), len(chars)), dtype=numpy.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 7478801
Unique characters: 128


In [6]:
len(chars)

128

In [7]:
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
metrics=['acc']
model.summary()

checkpoint_path = "training_3/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

early_stopping = tensorflow.keras.callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=10, # how many epochs to wait before stopping
    restore_best_weights=True,
)

# Create a callback that saves the model's weights
cp_callback = tensorflow.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1)

tensorboard = tensorflow.keras.callbacks.TensorBoard(
    log_dir='my_log_dir',
    histogram_freq=1,
    embeddings_freq=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 128)               16512     
Total params: 148,096
Trainable params: 148,096
Non-trainable params: 0
_________________________________________________________________


In [8]:
def sample(preds, temperature=1.0):
    preds = numpy.asarray(preds).astype('float64')
    preds = numpy.log(preds) / temperature
    exp_preds = numpy.exp(preds)
    preds = exp_preds / numpy.sum(exp_preds)
    probas = numpy.random.multinomial(1, preds, 1)
    return numpy.argmax(probas)

In [ ]:
model.fit(x, y,
          batch_size = 128,
          epochs = 10,
          callbacks=[cp_callback, tensorboard, early_stopping])

Epoch 1/10
    1/58429 [..............................] - ETA: 0s - loss: 4.8674WARNING:tensorflow:From C:\Users\andre\anaconda3\envs\bot\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
13840/58429 [======>.......................] - ETA: 1:25:46 - loss: 2.2687

In [61]:
start_index = random.randint(0, len(generalText) - maxlen - 1)
generated_text = generalText[start_index: start_index + maxlen]
print('\n---Generating with seed: ' + generated_text)
for temperature in [0.2, 0.5, 1.0, 1.2]:
    print('\n---Temperature: ' + str(temperature))
    sys.stdout.write(generated_text)
    for i in range(400):
        sampled = numpy.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.

        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]

        generated_text += next_char
        generated_text = generated_text[1:]

        sys.stdout.write(next_char)





---Generating with seed: 10 раз большую стоимости наших кораблей можно даже не писать

---Temperature: 0.2
10 раз большую стоимости наших кораблей можно даже не писать наслилькикакикликизррзрббовс бертвшеть га на нейнайна взо пртет е беель у притим сигац ну берна посто воерул я кекакаки зас рашиз берида фифмаль с стисешь иказа за в вворовы кокккккккк нней бдешше пукучисе преваля я во  зилочко всть это эо что протом моем флллина за драто пожание  прудаю илиху чис калоль ортосточтсей это почтол часл ру тобкаль 20 ститвини икак к пробше преват провиили я куткак ка
---Temperature: 0.5
обкаль 20 ститвини икак к пробше преват провиили я куткак какце игде ны какац поререй бдедать ны танной в далля не дроше  кукалим зак мацопн у го- провтонеелю)  мене них е бккава срегно и ненеть ппоссе просто у кака пры тс и конииячись гаку ум чем рвне о попасто того к лфлрашши фзрпзибы о пворятвине не прешнаши) изалихзих киззазарводо утум тешили на белкитс чис овпоров это млелуму я ттти иулл я к

KeyboardInterrupt: 

In [62]:
maxlen

60